In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_Key from the env file
load_dotenv()
api_key = os.getenv('NASA_API_Key')

In [ ]:
type(api_key)

In [ ]:
# Set the base URL to NASA's DONKI API:
# Set the specifier for CMEs:
cme_base_url = "https://api.nasa.gov/DONKI/CME"

# Search for CMEs published between a begin and end date
start_date = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for CME
query_url = f"{cme_base_url}?startDate={start_date}&endDate={end_date}&api_key={api_key}"

In [ ]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url)

In [ ]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [ ]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json, indent=4))

In [ ]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
cme_df.head()

In [ ]:
cme_df.linkedEvents.info()

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs

# Replace "None" with NaN
cme_df = cme_df.replace("None", pd.NA)  

# Drop all rows with NaN values
cme_df = cme_df.dropna()
cme_df.head()

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    activityID       = cme_df.loc[i, 'activityID']    # Get the corresponding value from 'gstID'
    startTime        = cme_df.loc[i, 'startTime']     # Get the corresponding value from 'startTime'    
    linkedEvents     = cme_df.loc[i, 'linkedEvents']  # Get the list of dictionaries in 'linkedEvents'

    try:
        # Iterate over each dictionary in the list
        for item in linkedEvents:
            # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
            expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': item})
    except:
        print('missing activity')

# Create a new DataFrame from the expanded rows
cme_expanded_rows_df = pd.DataFrame(expanded_rows)
cme_expanded_rows_df.head()

In [ ]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        activityID = input_dict.get('activityID', None)
        return activityID
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error processing input dictionary: {input_dict}. Error: {e}")
        return None

extract_activityID_from_dict(cme_expanded_rows_df.loc[0,'linkedEvents'])

In [ ]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_expanded_rows_df['GST_ActivityID'] = cme_expanded_rows_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
cme_expanded_rows_df.head()

In [ ]:
cme_expanded_rows_df

In [ ]:
cme_expanded_rows_df.info()

In [ ]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:

In [ ]:
# print out the datatype of each column in this DataFrame:
cme_expanded_rows_df.info()

In [ ]:
# Convert the 'GST_ActivityID' column to string format 
cme_expanded_rows_df['GST_ActivityID'] = cme_expanded_rows_df['GST_ActivityID'].astype(str)

In [ ]:
# Convert startTime to datetime format  
cme_expanded_rows_df['startTime'] = pd.to_datetime(cme_expanded_rows_df['startTime'])

In [ ]:
# Rename startTime to startTime_CME and activityID to cmeID
cme_expanded_rows_df.rename(columns={'startTime': 'startTime_CME', "activityID": "cmeID"}, inplace=True)

In [ ]:
# Drop linkedEvents
cme_expanded_rows_df.drop(columns=['linkedEvents'], inplace=True)

In [ ]:
# Verify that all steps were executed correctly
cme_expanded_rows_df.info()

In [ ]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library. 
cme_expanded_rows_df = cme_expanded_rows_df[cme_expanded_rows_df['GST_ActivityID'].str.contains('GST', na=False)]
cme_expanded_rows_df.head()

In [ ]:
# Set the base URL to NASA's DONKI API:
# Set the specifier for Geomagnetic Storms (GST):
gst_base_url = "https://api.nasa.gov/DONKI/GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url = f"{gst_base_url}?startDate={start_date}&endDate={end_date}&api_key={api_key}"

In [ ]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url)

In [ ]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json, indent=4))

In [ ]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

In [ ]:
print(gst_df.columns.tolist())

In [ ]:
# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df.head()

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME

# Replace "None" with NaN
gst_df = gst_df.replace("None", pd.NA)  

# Drop all rows with NaN values
gst_df = gst_df.dropna()
gst_df.head()


In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
expanded_rows = []

# Iterate over each index in the DataFrame
for i in gst_df.index:
    gstID       = gst_df.loc[i, 'gstID']    # Get the corresponding value from 'gstID'
    startTime        = gst_df.loc[i, 'startTime']     # Get the corresponding value from 'startTime'    
    linkedEvents     = gst_df.loc[i, 'linkedEvents']  # Get the list of dictionaries in 'linkedEvents'

    try:
        # Iterate over each dictionary in the list
        for item in linkedEvents:
            # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'gstID' and 'startTime' value
            expanded_rows.append({'gstID': gstID, 'startTime': startTime, 'linkedEvents': item})
    except:
        print('missing activity')

# Create a new DataFrame from the expanded rows
gst_expanded_rows_df = pd.DataFrame(expanded_rows)
gst_expanded_rows_df.head()

In [ ]:
extract_activityID_from_dict(gst_expanded_rows_df.loc[0,'linkedEvents'])

In [ ]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_expanded_rows_df['CME_ActivityID'] = gst_expanded_rows_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
gst_expanded_rows_df.head()

In [ ]:
gst_expanded_rows_df.info()

In [ ]:
# Convert the 'CME_ActivityID' column to string format 
gst_expanded_rows_df['CME_ActivityID'] = gst_expanded_rows_df['CME_ActivityID'].astype(str)

In [ ]:
# Convert the 'gstID' column to string format 
gst_expanded_rows_df['gstID'] = gst_expanded_rows_df['gstID'].astype(str)

In [ ]:
# Convert startTime to datetime format  
gst_expanded_rows_df['startTime'] = pd.to_datetime(gst_expanded_rows_df['startTime'])

In [ ]:
# Rename startTime to startTime_GST 
gst_expanded_rows_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

In [ ]:
# Drop linkedEvents
gst_expanded_rows_df.drop(columns=['linkedEvents'], inplace=True)

In [ ]:
# Verify that all steps were executed correctly
gst_expanded_rows_df.info()

In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_expanded_rows_df = gst_expanded_rows_df[gst_expanded_rows_df['CME_ActivityID'].str.contains('CME', na=False)]
gst_expanded_rows_df.reset_index(drop=True, inplace=True)
gst_expanded_rows_df.head()

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_expanded_rows_df, cme_expanded_rows_df, 
                     left_on='CME_ActivityID', 
                     right_on='cmeID', 
                     how='inner')
merged_df.head()

In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.info()

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']
merged_df.head()

In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
merged_df['timeDiff'].describe()

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)